In [46]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

import pyspark.sql.types as t
import pyspark.sql.functions as f

from pyspark.sql.window import Window

import columns as c

In [2]:
MINIMAL_TRIP_TIME_SEC = 3*60
MINIMAL_FARE_AMOUNT = 3.0
NYC_TIMEZONE = "America/New_York"

TARGET_MONTH_NUM = 3 # march

MAX_PASSENGER_NUM = 4
MILES_SEC_TO_KM_HOUR = 5793.6384
STD_CITY_RATE = 1 # Standard City Rate

In [3]:
spark_session = (SparkSession.builder
                     .master('local')
                     .appName('taxiSparlApp')
                     .config(conf=SparkConf())
                     .getOrCreate())

In [4]:
DATA_ROOT_PATH = "path/to/data"
TRIP_DATA_PATH = DATA_ROOT_PATH + "/trip_data/trip_data_3.csv"
TRIP_DATA_PATH_ALL = [ 
    # DATA_ROOT_PATH + "/trip_data/trip_data_1.csv",
    # DATA_ROOT_PATH + "/trip_data/trip_data_2.csv",
    DATA_ROOT_PATH + "/trip_data/trip_data_3.csv",
    DATA_ROOT_PATH + "/trip_data/trip_data_4.csv"
    # DATA_ROOT_PATH + "/trip_data/trip_data_5.csv",
    # DATA_ROOT_PATH + "/trip_data/trip_data_6.csv",
    # DATA_ROOT_PATH + "/trip_data/trip_data_7.csv",
    # DATA_ROOT_PATH + "/trip_data/trip_data_8.csv",
    # DATA_ROOT_PATH + "/trip_data/trip_data_9.csv",
    # DATA_ROOT_PATH + "/trip_data/trip_data_10.csv",
    # DATA_ROOT_PATH + "/trip_data/trip_data_11.csv",
#    DATA_ROOT_PATH + "/trip_data/trip_data_12.csv"
]

TRIP_FARE_PATH = DATA_ROOT_PATH + "/trip_fare/trip_fare_1.csv"
TRIP_FARE_PATH_ALL = [ 
    # DATA_ROOT_PATH + "/trip_fare/trip_fare_1.csv",
    # DATA_ROOT_PATH + "/trip_fare/trip_fare_2.csv",
    DATA_ROOT_PATH + "/trip_fare/trip_fare_3.csv",
    DATA_ROOT_PATH + "/trip_fare/trip_fare_4.csv"
    # DATA_ROOT_PATH + "/trip_fare/trip_fare_5.csv",
    # DATA_ROOT_PATH + "/trip_fare/trip_fare_6.csv",
    # DATA_ROOT_PATH + "/trip_fare/trip_fare_7.csv",
    # DATA_ROOT_PATH + "/trip_fare/trip_fare_8.csv",
    # DATA_ROOT_PATH + "/trip_fare/trip_fare_9.csv",
    # DATA_ROOT_PATH + "/trip_fare/trip_fare_10.csv",
    # DATA_ROOT_PATH + "/trip_fare/trip_fare_11.csv",
    #DATA_ROOT_PATH + "/trip_fare/trip_fare_12.csv"
]



In [5]:
trip_schema = t.StructType([
    t.StructField("medallion", t.StringType(), True),
    t.StructField(" hack_license", t.StringType(), True),
    t.StructField(" vendor_id", t.StringType(), True),
    t.StructField(" rate_code", t.IntegerType(), True),
    t.StructField(" store_and_fwd_flag", t.StringType(), True),
    t.StructField(" pickup_datetime", t.TimestampType(), True),
    t.StructField(" dropoff_datetime", t.TimestampType(), True),
    t.StructField(" passenger_count", t.IntegerType(), True),
    t.StructField(" trip_time_in_secs", t.StringType(), True),
    t.StructField(" trip_distance", t.FloatType(), True),
    t.StructField(" pickup_longitude", t.FloatType(), True),
    t.StructField(" pickup_latitude", t.FloatType(), True),
    t.StructField(" dropoff_longitude", t.FloatType(), True),
    t.StructField(" dropoff_latitude", t.FloatType(), True)
])


In [6]:
fare_schema = t.StructType([
    t.StructField("medallion", t.StringType(), True),
    t.StructField(" hack_license", t.StringType(), True),
    t.StructField(" vendor_id", t.StringType(), True),
    t.StructField(" pickup_datetime", t.TimestampType(), True),
    t.StructField(" payment_type", t.StringType(), True),
    t.StructField(" fare_amount", t.FloatType(), True),
    t.StructField(" surcharge", t.FloatType(), True),
    t.StructField(" mta_tax", t.FloatType(), True),
    t.StructField(" tip_amount", t.FloatType(), True),
    t.StructField(" tolls_amount", t.FloatType(), True),
    t.StructField(" total_amount", t.FloatType(), True)
])

In [7]:
def column_names_list(df):
    col_names = [col.lower().replace(" ", "_") for col in df.columns]
    for i in range(len(col_names)):
        print(df.columns[i],"=", f"'{col_names[i]}'")

In [8]:
def strip_columns(df):
    new_col_names =  [col.strip() for col in df.columns]
    new_column_name_list= list(map(lambda x: x.strip(), df.columns))
    df = df.toDF(*new_column_name_list)
    return df

In [9]:
trip_df_raw = spark_session.read.csv(TRIP_DATA_PATH_ALL, sep=',',header=True, schema=trip_schema)

# trip_df_raw.schema
trip_df_raw.show()

+--------------------+--------------------+----------+----------+-------------------+-------------------+-------------------+----------------+------------------+--------------+-----------------+----------------+------------------+-----------------+
|           medallion|        hack_license| vendor_id| rate_code| store_and_fwd_flag|    pickup_datetime|   dropoff_datetime| passenger_count| trip_time_in_secs| trip_distance| pickup_longitude| pickup_latitude| dropoff_longitude| dropoff_latitude|
+--------------------+--------------------+----------+----------+-------------------+-------------------+-------------------+----------------+------------------+--------------+-----------------+----------------+------------------+-----------------+
|FE7B354FEB67B9C94...|2C78614ADC9C602EC...|       CMT|         1|                  N|2013-03-01 00:00:04|2013-03-01 00:19:03|               1|              1138|          14.3|         -73.7767|       40.645164|        -73.913925|        40.772614|
|B13

In [10]:
fare_df_raw = spark_session.read.csv(TRIP_FARE_PATH_ALL, sep=',',header=True, schema=fare_schema)

# fare_df_raw.schema
fare_df_raw.show()

+--------------------+--------------------+----------+-------------------+-------------+------------+----------+--------+-----------+-------------+-------------+
|           medallion|        hack_license| vendor_id|    pickup_datetime| payment_type| fare_amount| surcharge| mta_tax| tip_amount| tolls_amount| total_amount|
+--------------------+--------------------+----------+-------------------+-------------+------------+----------+--------+-----------+-------------+-------------+
|FE7B354FEB67B9C94...|2C78614ADC9C602EC...|       CMT|2013-03-01 00:00:04|          CRD|        39.5|       0.5|     0.5|       10.1|          0.0|         50.6|
|B1370E6501ED61CED...|08BF18740B38D6472...|       CMT|2013-03-01 00:00:18|          CRD|        13.0|       0.5|     0.5|        2.1|          0.0|         16.1|
|CBC20164C22B96CAF...|F3310C8DA3DB753DD...|       CMT|2013-03-01 00:00:18|          CRD|        15.0|       0.5|     0.5|        3.2|          0.0|         19.2|
|B8256802A449BEF34...|DF142C

In [11]:
trip_df_raw = strip_columns(trip_df_raw)
trip_df_raw.columns

['medallion',
 'hack_license',
 'vendor_id',
 'rate_code',
 'store_and_fwd_flag',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_time_in_secs',
 'trip_distance',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude']

In [12]:
fare_df_raw = strip_columns(fare_df_raw)
fare_df_raw.columns

['medallion',
 'hack_license',
 'vendor_id',
 'pickup_datetime',
 'payment_type',
 'fare_amount',
 'surcharge',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'total_amount']

In [13]:
# skip

column_names_list(trip_df_raw)

medallion = 'medallion'
hack_license = 'hack_license'
vendor_id = 'vendor_id'
rate_code = 'rate_code'
store_and_fwd_flag = 'store_and_fwd_flag'
pickup_datetime = 'pickup_datetime'
dropoff_datetime = 'dropoff_datetime'
passenger_count = 'passenger_count'
trip_time_in_secs = 'trip_time_in_secs'
trip_distance = 'trip_distance'
pickup_longitude = 'pickup_longitude'
pickup_latitude = 'pickup_latitude'
dropoff_longitude = 'dropoff_longitude'
dropoff_latitude = 'dropoff_latitude'


In [14]:
# skip

column_names_list(fare_df_raw)

medallion = 'medallion'
hack_license = 'hack_license'
vendor_id = 'vendor_id'
pickup_datetime = 'pickup_datetime'
payment_type = 'payment_type'
fare_amount = 'fare_amount'
surcharge = 'surcharge'
mta_tax = 'mta_tax'
tip_amount = 'tip_amount'
tolls_amount = 'tolls_amount'
total_amount = 'total_amount'


In [15]:
trip_columns_to_drop = [
    c.store_and_fwd_flag,
    c.dropoff_datetime,
    c.pickup_longitude,
    c.pickup_latitude,
    c.dropoff_longitude,
    c.dropoff_latitude
]
trip_df_raw = trip_df_raw.drop(*trip_columns_to_drop)
trip_df_raw.columns

['medallion',
 'hack_license',
 'vendor_id',
 'rate_code',
 'pickup_datetime',
 'passenger_count',
 'trip_time_in_secs',
 'trip_distance']

In [16]:
fare_columns_to_drop = [
    c.tolls_amount,
    c.mta_tax,
]
fare_df_raw = fare_df_raw.drop(*fare_columns_to_drop)
fare_df_raw.columns

['medallion',
 'hack_license',
 'vendor_id',
 'pickup_datetime',
 'payment_type',
 'fare_amount',
 'surcharge',
 'tip_amount',
 'total_amount']

In [17]:
trip_df_raw.count()

30849696

In [18]:
fare_df_raw.count()

30849696

In [19]:
#skip 
trip_df_raw.select([f.count(f.when(f.col(c).isNull(), c)).alias(c) for c in trip_df_raw.columns]
   ).show()

+---------+------------+---------+---------+---------------+---------------+-----------------+-------------+
|medallion|hack_license|vendor_id|rate_code|pickup_datetime|passenger_count|trip_time_in_secs|trip_distance|
+---------+------------+---------+---------+---------------+---------------+-----------------+-------------+
|        0|           0|        0|        0|              0|              0|                0|            0|
+---------+------------+---------+---------+---------------+---------------+-----------------+-------------+



In [20]:
#skip 
fare_df_raw.select([f.count(f.when(f.col(c).isNull(), c)).alias(c) for c in fare_df_raw.columns]
   ).show()

+---------+------------+---------+---------------+------------+-----------+---------+----------+------------+
|medallion|hack_license|vendor_id|pickup_datetime|payment_type|fare_amount|surcharge|tip_amount|total_amount|
+---------+------------+---------+---------------+------------+-----------+---------+----------+------------+
|        0|           0|        0|              0|           0|          0|        0|         0|           0|
+---------+------------+---------+---------------+------------+-----------+---------+----------+------------+



In [21]:
trip_df = trip_df_raw.filter(
    (f.month(c.pickup_datetime) == TARGET_MONTH_NUM) 
)

In [22]:
fare_df = fare_df_raw.filter(
    (f.month(c.pickup_datetime) == TARGET_MONTH_NUM) 
)

In [23]:
trip_df.select(c.rate_code, c.passenger_count, c.trip_time_in_secs, c.trip_distance).summary('min', "mean", 'max').show()

+-------+------------------+------------------+-----------------+------------------+
|summary|         rate_code|   passenger_count|trip_time_in_secs|     trip_distance|
+-------+------------------+------------------+-----------------+------------------+
|    min|                 0|                 0|                0|               0.0|
|   mean|1.0338665488873486|1.7121524305826292|717.4479485597643|2.8414825427954855|
|    max|               210|               255|              999|             100.0|
+-------+------------------+------------------+-----------------+------------------+



In [24]:
fare_df.select(c.fare_amount, c.total_amount).summary('min', "mean", 'max').show()

+-------+-----------------+------------------+
|summary|      fare_amount|      total_amount|
+-------+-----------------+------------------+
|    min|              2.5|               2.5|
|   mean|12.03511757978071|14.395476985343125|
|    max|            500.0|             529.8|
+-------+-----------------+------------------+



In [25]:
trip_df = trip_df.filter(
    (trip_df_raw.trip_time_in_secs >= MINIMAL_TRIP_TIME_SEC)
    & (trip_df_raw.rate_code == STD_CITY_RATE)
    & (trip_df.passenger_count <= MAX_PASSENGER_NUM)
)

In [26]:
fare_df = fare_df.filter(
    (fare_df_raw.total_amount >= MINIMAL_FARE_AMOUNT)
)

In [27]:
taxi_df = trip_df.join(fare_df, on=[c.medallion, c.hack_license, c.vendor_id, c.pickup_datetime])

taxi_df.cache()
# joined_df.show()

DataFrame[medallion: string, hack_license: string, vendor_id: string, pickup_datetime: timestamp, rate_code: int, passenger_count: int, trip_time_in_secs: string, trip_distance: float, payment_type: string, fare_amount: float, surcharge: float, tip_amount: float, total_amount: float]

In [28]:
taxi_df = taxi_df.drop(c.rate_code)
taxi_df = taxi_df.withColumn(c.pickup_datetime, f.from_utc_timestamp(c.pickup_datetime, NYC_TIMEZONE))
taxi_df = taxi_df.withColumn("avg_speed", ((f.col(c.trip_distance) / f.col(c.trip_time_in_secs)) * MILES_SEC_TO_KM_HOUR))
taxi_df = taxi_df.drop(c.trip_distance, c.trip_time_in_secs)

taxi_df.cache()

taxi_df.show()

+--------------------+--------------------+---------+-------------------+---------------+------------+-----------+---------+----------+------------+------------------+
|           medallion|        hack_license|vendor_id|    pickup_datetime|passenger_count|payment_type|fare_amount|surcharge|tip_amount|total_amount|         avg_speed|
+--------------------+--------------------+---------+-------------------+---------------+------------+-----------+---------+----------+------------+------------------+
|00005007A9F30E289...|16780B3E72BAA7A5C...|      CMT|2013-03-27 16:57:23|              1|         CRD|       12.5|      0.5|      4.05|       17.55| 18.36235181696657|
|00005007A9F30E289...|39C3E34B3E338A721...|      CMT|2013-03-17 08:30:49|              1|         CRD|       10.5|      0.0|       2.0|        13.0| 20.18103441585546|
|00005007A9F30E289...|39C3E34B3E338A721...|      CMT|2013-03-17 12:48:18|              1|         CRD|        5.0|      0.0|       1.0|         6.5|16.157557015

In [29]:
taxi_df.count()

13200950

In [38]:
taxi_df.select(c.vendor_id, c.pickup_datetime, c.payment_type, c.fare_amount, c.tip_amount).show()

+---------+-------------------+------------+-----------+----------+
|vendor_id|    pickup_datetime|payment_type|fare_amount|tip_amount|
+---------+-------------------+------------+-----------+----------+
|      CMT|2013-03-27 16:57:23|         CRD|       12.5|      4.05|
|      CMT|2013-03-17 08:30:49|         CRD|       10.5|       2.0|
|      CMT|2013-03-17 12:48:18|         CRD|        5.0|       1.0|
|      CMT|2013-03-19 04:29:05|         CRD|       17.0|       1.5|
|      CMT|2013-03-05 03:25:42|         CRD|        8.5|       1.0|
|      CMT|2013-03-15 20:37:17|         CRD|       18.0|       3.8|
|      CMT|2013-03-21 20:28:18|         CSH|       15.5|       0.0|
|      CMT|2013-03-25 17:57:14|         CRD|       11.5|       2.5|
|      VTS|2013-03-31 14:40:00|         CRD|        8.5|       1.0|
|      VTS|2013-03-03 15:16:00|         CRD|       14.0|       2.9|
|      VTS|2013-03-21 19:04:00|         CRD|       11.0|       2.0|
|      CMT|2013-03-07 12:48:50|         CRD|    

In [40]:
grouped_df = taxi_df.groupBy("vendor_id", "payment_type").agg(
    f.round(f.avg("fare_amount"),5).alias("avg_fare_amount"),
    f.round(f.avg("tip_amount"),5).alias("avg_tip_amount"),
    f.count("*").alias("row_count")
)

grouped_df.show()

+---------+------------+---------------+--------------+---------+
|vendor_id|payment_type|avg_fare_amount|avg_tip_amount|row_count|
+---------+------------+---------------+--------------+---------+
|      CMT|         DIS|       14.23206|       0.01464|     7830|
|      CMT|         CSH|       10.86614|       0.00142|  3387347|
|      CMT|         CRD|       12.04952|       2.36257|  3881217|
|      VTS|         CRD|       12.04416|       2.27002|  3199127|
|      CMT|         NOC|       12.97715|        0.0057|    19830|
|      VTS|         UNK|       13.33046|       2.75677|     5161|
|      VTS|         CSH|       10.84117|        8.0E-5|  2700438|
+---------+------------+---------------+--------------+---------+



In [42]:

window_spec_fare = Window.partitionBy(c.vendor_id).orderBy(f.desc("avg_fare_amount"))
window_spec_tip = Window.partitionBy(c.vendor_id).orderBy(f.desc("avg_tip_amount"))

ranked_df = grouped_df.withColumn("fare_amount_rank", f.rank().over(window_spec_fare))
ranked_df = ranked_df.withColumn("tip_amount_rank", f.rank().over(window_spec_tip)).orderBy("fare_amount_rank","tip_amount_rank")

ranked_df.show()

+---------+------------+---------------+--------------+---------+----------------+---------------+
|vendor_id|payment_type|avg_fare_amount|avg_tip_amount|row_count|fare_amount_rank|tip_amount_rank|
+---------+------------+---------------+--------------+---------+----------------+---------------+
|      VTS|         UNK|       13.33046|       2.75677|     5161|               1|              1|
|      CMT|         DIS|       14.23206|       0.01464|     7830|               1|              2|
|      VTS|         CRD|       12.04416|       2.27002|  3199127|               2|              2|
|      CMT|         NOC|       12.97715|        0.0057|    19830|               2|              3|
|      CMT|         CRD|       12.04952|       2.36257|  3881217|               3|              1|
|      VTS|         CSH|       10.84117|        8.0E-5|  2700438|               3|              3|
|      CMT|         CSH|       10.86614|       0.00142|  3387347|               4|              4|
+---------

In [43]:
ranked_df.filter(f.col(c.payment_type).isin(["CRD", "CSH"])).show()

+---------+------------+---------------+--------------+---------+----------------+---------------+
|vendor_id|payment_type|avg_fare_amount|avg_tip_amount|row_count|fare_amount_rank|tip_amount_rank|
+---------+------------+---------------+--------------+---------+----------------+---------------+
|      VTS|         CRD|       12.04416|       2.27002|  3199127|               2|              2|
|      CMT|         CRD|       12.04952|       2.36257|  3881217|               3|              1|
|      VTS|         CSH|       10.84117|        8.0E-5|  2700438|               3|              3|
|      CMT|         CSH|       10.86614|       0.00142|  3387347|               4|              4|
+---------+------------+---------------+--------------+---------+----------------+---------------+



In [32]:
taxi_df.select(
  f.countDistinct(c.medallion).alias("medallion"),
  f.countDistinct(c.hack_license).alias("hack_license")
).show()


+---------+------------+
|medallion|hack_license|
+---------+------------+
|    13274|       31631|
+---------+------------+



In [39]:
taxi_df.select(c.payment_type).distinct().show()

+------------+
|payment_type|
+------------+
|         CSH|
|         DIS|
|         CRD|
|         UNK|
|         NOC|
+------------+



In [53]:
taxi_df.select(c.pickup_datetime, c.fare_amount, c.surcharge, 
               c.tip_amount, c.total_amount, 
               f.round("avg_speed", 3).alias("avg_speed")).show()

+-------------------+-----------+---------+----------+------------+---------+
|    pickup_datetime|fare_amount|surcharge|tip_amount|total_amount|avg_speed|
+-------------------+-----------+---------+----------+------------+---------+
|2013-03-27 16:57:23|       12.5|      0.5|      4.05|       17.55|   18.362|
|2013-03-17 08:30:49|       10.5|      0.0|       2.0|        13.0|   20.181|
|2013-03-17 12:48:18|        5.0|      0.0|       1.0|         6.5|   16.158|
|2013-03-19 04:29:05|       17.0|      0.0|       1.5|        19.0|   15.129|
|2013-03-05 03:25:42|        8.5|      0.0|       1.0|        10.0|   13.189|
|2013-03-15 20:37:17|       18.0|      0.5|       3.8|        22.8|   25.279|
|2013-03-21 20:28:18|       15.5|      0.5|       0.0|        16.5|     34.6|
|2013-03-25 17:57:14|       11.5|      0.5|       2.5|        15.0|   38.546|
|2013-03-31 14:40:00|        8.5|      0.0|       1.0|        10.0|   22.692|
|2013-03-03 15:16:00|       14.0|      0.5|       2.9|        17

In [35]:
average_tips_by_weekday = taxi_df.groupBy(f.dayofweek(c.pickup_datetime).alias("day_idx")) \
     .agg(
         f.count("*").alias("row_count"),
         f.round(f.mean(c.fare_amount),5).alias("avg_fare_amount"),
         f.round(f.stddev(c.fare_amount),5).alias("stddev_fare_amount"),
         f.round(f.mean(c.surcharge),5).alias("avg_surcharge"),
         f.round(f.mean(c.tip_amount),5).alias("avg_tip_amount"),
         f.round(f.mean(c.total_amount),5).alias("avg_total_amount"),
        
    ).orderBy("day_idx")

average_tips_by_weekday = average_tips_by_weekday.withColumn('weekday',
                                                             f.when(f.col("day_idx") == 1, "Sunday")
                                                             .when(f.col("day_idx") == 2, "Monday")
                                                             .when(f.col("day_idx") == 3, "Tuesday")
                                                             .when(f.col("day_idx") == 4, "Wednesday")
                                                             .when(f.col("day_idx") == 5, "Thursday")
                                                             .when(f.col("day_idx") == 6, "Friday")
                                                             .when(f.col("day_idx") == 7, "Saturday"))
average_tips_by_weekday.show()

+-------+---------+---------------+------------------+-------------+--------------+----------------+---------+
|day_idx|row_count|avg_fare_amount|stddev_fare_amount|avg_surcharge|avg_tip_amount|avg_total_amount|  weekday|
+-------+---------+---------------+------------------+-------------+--------------+----------------+---------+
|      1|  1572669|       11.65321|           7.73176|      0.14364|       1.15518|        13.64748|   Sunday|
|      2|  1521814|       11.28764|            7.5294|      0.37784|       1.26842|        13.62595|   Monday|
|      3|  1642473|       11.35368|            7.2596|       0.3717|       1.30146|        13.69226|  Tuesday|
|      4|  1712867|       11.46297|           7.26995|      0.37632|       1.32575|        13.83595|Wednesday|
|      5|  1877892|       11.69795|           7.49998|      0.37654|        1.3155|        14.06378| Thursday|
|      6|  2507065|       11.58422|           7.34769|      0.39923|       1.26173|        13.89639|   Friday|
|

In [54]:
average_tips_by_hour = taxi_df.groupBy(f.hour(c.pickup_datetime).alias("hour")) \
                                .agg(
                                    f.round(f.count("*"),5).alias("row_count"),
                                    f.round(f.mean("avg_speed"),5).alias("avg_speed"), 
                                    f.round(f.mean(c.fare_amount),5).alias("avg_fare_amount"),
                            
                                    f.round(f.mean(c.surcharge),5).alias("avg_surcharge"),
                                   
                                    f.round(f.mean(c.tip_amount),5).alias("avg_tip_amount"),
                                   
                                    f.round(f.mean(c.total_amount),5).alias("avg_total_amount"),
                                ).orderBy("hour")

average_tips_by_hour.show(24)

+----+---------+---------+---------------+-------------+--------------+----------------+
|hour|row_count|avg_speed|avg_fare_amount|avg_surcharge|avg_tip_amount|avg_total_amount|
+----+---------+---------+---------------+-------------+--------------+----------------+
|   0|   132634| 31.83094|       14.10929|      0.49386|       1.31163|        16.70518|
|   1|   152571|  30.5245|       12.62722|      0.22616|       1.28523|        14.97216|
|   2|   307041| 25.17298|       11.17498|      0.00161|       1.18329|        13.07046|
|   3|   474100| 20.91488|       11.03072|       4.4E-4|       1.24401|        12.94664|
|   4|   576827| 18.21941|       11.20746|       4.0E-4|       1.29958|        13.16561|
|   5|   608655| 18.12692|       11.28717|       4.2E-4|       1.26393|        13.23533|
|   6|   606650| 18.30195|       11.21618|       4.0E-4|       1.17188|        13.07285|
|   7|   631315| 17.87479|       11.21528|       3.9E-4|        1.1536|           13.05|
|   8|   650871| 17.7